# Pneumonia CNN Baseline
Clean multi‑cell notebook version

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

SEED = 42
tf.random.set_seed(SEED)

DATA_DIR = "chest_xray"
IMG_SIZE = (160, 160)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

train_dir = os.path.join(DATA_DIR, "train")
val_dir   = os.path.join(DATA_DIR, "val")
test_dir  = os.path.join(DATA_DIR, "test")

## Load datasets

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='binary',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='binary',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='binary',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

class_names = train_ds.class_names
class_names

In [ ]:
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds   = val_ds.prefetch(AUTOTUNE)
test_ds  = test_ds.prefetch(AUTOTUNE)

## Data augmentation

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
])

## Build CNN model

In [ ]:
def build_cnn_model(
    input_shape=IMG_SIZE + (3,),
    num_filters=(16, 32, 64),
    dense_units=64,
    dropout_rate=0.5,
):
    inputs = keras.Input(shape=input_shape)

    x = data_augmentation(inputs)
    x = layers.Rescaling(1.0/255)(x)

    for f in num_filters:
        x = layers.Conv2D(f, 3, padding='same', activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D()(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(dense_units, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)

    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy', keras.metrics.AUC(), keras.metrics.Precision(), keras.metrics.Recall()]
    )
    return model

model = build_cnn_model()
model.summary()

## Class weights

In [ ]:
normal_count = 1341
pneu_count = 3875
total = normal_count + pneu_count

class_weight = {
    0: total / (2 * normal_count),
    1: total / (2 * pneu_count),
}
class_weight

## Callbacks

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    keras.callbacks.ModelCheckpoint('best_pneumonia_cnn.keras', save_best_only=True),
]

## Train

In [ ]:
EPOCHS = 10

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    class_weight=class_weight,
    callbacks=callbacks,
)

## Evaluate on test set

In [ ]:
test_metrics = model.evaluate(test_ds)
list(zip(model.metrics_names, test_metrics))

Quick test: comparing different CNN depths

This section runs a small comparison to see how the number of conv–pool blocks affects model performance. All other settings (optimizer, augmentation, dropout, batch size, etc.) are kept the same so that depth is the only variable being changed.

The three architectures tested are:

1 block: (32,) – very shallow

2 blocks: (32, 64) – lightweight baseline

3 blocks: (32, 64, 128) – slightly deeper

Since the aim is just to compare behaviour rather than fully train each model, the runs are limited to 5 epochs with EarlyStopping enabled. The key things being observed are:

differences in validation accuracy and validation loss

whether deeper models begin to overfit sooner

the change in training time per epoch across depths

The depth that performs best here will be used as the recommended architecture for the final combined model later in the project.